# Intro

## Load

### Standards

In [1]:
import os, sys, pickle
import numpy as np
from tqdm.notebook import trange, tqdm
import re

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.major.width'] = 2

plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1


### Homemade modules

In [3]:
from tesste import tesste
from ppp import ppp
from readers_and_converters import file2text, text2dict, text2vec

## Stoopid infos

In [4]:
L_SDGS=17

# Data

In [5]:
with open ('./data/energy_utilities_texts.pickle', 'rb') as f:
    text_dict=pickle.load(f)

# Semantic analysis

In [ ]:
cos_sim_dict={}
for year in range(2015, 2024):
    s_year=str(year)
    cos_sim_dict[year]={}
    energy_co_s=list(text_dict['Energy'][s_year].keys())
    # inside energy
    energy_cos_sims=np.zeros((len(energy_co_s), len(energy_co_s)))
    energy_pvals=np.zeros((len(energy_co_s), len(energy_co_s)))
    for i_e, energy_co_0 in enumerate(tqdm(energy_co_s, leave=False, desc='Energy '+s_year)):
        text_0=text_dict['Energy'][s_year][energy_co_0]
        for j_e in range(i_e+1, len(energy_co_s)):
            energy_co_1=energy_co_s[j_e]
            text_1=text_dict['Energy'][s_year][energy_co_1]
            cacca=tesste(text_0, text_1)
            energy_cos_sims[i_e, j_e]=cacca.cos_sim
            energy_pvals[i_e, j_e]=cacca.pval
            
    cos_sim_dict[year]['Energy']={}
    cos_sim_dict[year]['Energy']['cos_sim']=energy_cos_sims
    cos_sim_dict[year]['Energy']['pvals']=energy_pvals
    
    utilities_co_s=list(text_dict['Utilities'][s_year].keys())
    utilities_cos_sims=np.zeros((len(utilities_co_s), len(utilities_co_s)))
    utilities_pvals=np.zeros((len(utilities_co_s), len(utilities_co_s)))
    # inside utilities
    for i_e, utilities_co_0 in enumerate(tqdm(utilities_co_s, leave=False, desc='Utilities '+s_year)):
        text_0=text_dict['Utilities'][s_year][utilities_co_0]
        for j_e in range(i_e+1, len(utilities_co_s)):
            utilities_co_1=utilities_co_s[j_e]
            text_1=text_dict['Utilities'][s_year][utilities_co_1]
            cacca=tesste(text_0, text_1)
            utilities_cos_sims[i_e, j_e]=cacca.cos_sim
            utilities_pvals[i_e, j_e]=cacca.pval
            
    cos_sim_dict[year]['Utilities']={}
    cos_sim_dict[year]['Utilities']['cos_sim']=utilities_cos_sims
    cos_sim_dict[year]['Utilities']['pvals']=utilities_pvals


Energy 2015:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
with open ('./data/energy_utilities_cos_sim_mats.pickle', 'wb') as f:
    pickle.dump(cos_sim_dict, f)